### `Homework_Chapter 2: Regression`

### Dataset

In this homework, we will use the Car price dataset. Download it from [here](https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### **Getting the data**

In [5]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

--2023-10-04 15:10:47--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv.1’

data.csv.1          100%[===================>]   1.41M  6.15MB/s    in 0.2s    

2023-10-04 15:10:48 (6.15 MB/s) - ‘data.csv.1’ saved [1475504/1475504]



### Question 1

What is the most frequent observation (mode) for the column `transmission_type`?

- `AUTOMATIC`
- `MANUAL`
- `AUTOMATED_MANUAL`
- `DIRECT_DRIVE`

In [71]:
df_car = pd.read_csv('data.csv')
df_car.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [72]:
df_car.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [8]:
# Column name transformation/cleaning
df_car.columns = df_car.columns.str.lower().str.replace(' ', '_')
df_car.columns

Index(['make', 'model', 'year', 'engine_fuel_type', 'engine_hp',
       'engine_cylinders', 'transmission_type', 'driven_wheels',
       'number_of_doors', 'market_category', 'vehicle_size', 'vehicle_style',
       'highway_mpg', 'city_mpg', 'popularity', 'msrp'],
      dtype='object')

In [10]:
q1 = df_car['transmission_type'].mode()[0]
print(f'Q1 Answer: {q1}')

Q1 Answer: AUTOMATIC


### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?

- `engine_hp` and `year`
- `engine_hp` and `engine_cylinders`
- `highway_mpg` and `engine_cylinders`
- `highway_mpg` and `city_mpg`



In [12]:
# Creating categorical and numerical columns list
categorical = ['make', 'model', 'transmission_type', 'vehicle_style']
numerical = ['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']

In [13]:
# Calculating the correlation matrix
correlation_matrix = df_car[numerical].corr()
correlation_matrix

# Find 2 features with the highes correlation
max_cor = correlation_matrix.abs().unstack()
max_cor = max_cor[max_cor !=1].idxmax()


feature1, feature2 = max_cor

print(f'Q2 Answer: {feature1} and {feature2}')

Q2 Answer: highway_mpg and city_mpg



### Make `price` binary

* Now we need to turn the `price` variable from numeric into a binary format.
* Let's create a variable `above_average` which is `1` if the `price` is above its mean value and `0` otherwise.


In [15]:
# Rename MSRP to price (use orig_data df)
df_car.rename(columns={'msrp':'price'}, inplace=True)
price_col = df_car['price']
price_col.head()

0    46135
1    40650
2    36350
3    29450
4    34500
Name: price, dtype: int64

In [17]:
# Selecting needed columns
new_car = df_car[['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style',
       'highway_mpg', 'city_mpg']]

In [19]:
# Replacing null values with 0
new_car = new_car.fillna(0)
new_car.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
above_average        0
dtype: int64

In [22]:
# Create the "above_average" column based on the condition
new_car['above_average'] = (price_col > price_col.mean()).astype(int)


### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value (`above_average`) is not in your dataframe.

In [23]:
from sklearn.model_selection import train_test_split

# Define features and target
X = new_car.drop('above_average', axis = 1)
y = new_car['above_average']

In [24]:
# Split the data into train, validation and test sets
# Use a 60%/20%/20% distribution with a random seed of 42
X_train, X_temp, y_train, y_temp = train_test_split(X,y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [25]:
# Check shapes
print(f'Train Set shape: {X_train.shape}')
print(f'Valication Set shape: {X_val.shape}')
print(f'Test Set shape: {X_test.shape}')

Train Set shape: (7148, 9)
Valication Set shape: (2383, 9)
Test Set shape: (2383, 9)



### Question 3

* Calculate the mutual information score between `above_average` and other categorical variables in our dataset.
  Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the lowest mutual information score?
  
- `make`
- `model`
- `transmission_type`
- `vehicle_style`

In [32]:
from sklearn.metrics import mutual_info_score

mutualinfo_scores = {}

# Calculating mutual infromation scores per categorical feature
for feature in categorical:
    mutualinfo_score = mutual_info_score(X_train[feature], y_train)
    mutualinfo_scores[feature] = round(mutualinfo_score, 2)

In [34]:
# Sorting the dictionary
sorted_mi_scores = dict(sorted(mutualinfo_scores.items(), key=lambda x: x[1], reverse=True))
for feature, score in sorted_mi_scores.items():
    print(f'{feature}:{score}')

model:0.46
make:0.24
vehicle_style:0.08
transmission_type:0.02


In [35]:
# other method (same with the video)
from sklearn.metrics import mutual_info_score

def mutual_info_ave_score(series):
    mis = mutual_info_score(series, y_train)
    return round(mis,2)

mutualinfo_scores = X_train[categorical].apply(mutual_info_ave_score)
mutualinfo_scores.sort_values(ascending=False)

model                0.46
make                 0.24
vehicle_style        0.08
transmission_type    0.02
dtype: float64

In [36]:
mutualinfo_lowest = list(sorted_mi_scores.keys())[-1]

In [37]:
print(f'Q3 Answer: {mutualinfo_lowest}')

Q3 Answer: transmission_type



### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.60
- 0.72
- 0.84
- 0.95


In [38]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [39]:
# Creating one-hot encoding
preprocessor = ColumnTransformer(
transformers=[('cat', OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'), categorical)], remainder='passthrough')


In [40]:
# Creating logistic Regression
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

In [41]:
# Creating a pipeline & data preprocessing and model fitting
pipeline = Pipeline([('preprocessor', preprocessor), ('model', model)])

In [42]:
# Combining training and validation data for one-hot encoding
X_combined = pd.concat([X_train, X_val], axis=0)
y_combined = pd.concat([y_train, y_val], axis=0)

In [43]:
# Fit the model on the training set
pipeline.fit(X_combined, y_combined)

# Predict on the validation dataset
y_val_pred = pipeline.predict(X_val)

# Calculating the accuracy on the validation dataset (rounded to 2 decimal places)
accuracy = round(accuracy_score(y_val, y_val_pred), 2)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [44]:
print(f'Q4 Answer: {accuracy}')

Q4 Answer: 0.94



### Question 5

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

- `year`
- `engine_hp`
- `transmission_type`
- `city_mpg`

> **Note**: the difference doesn't have to be positive

In [67]:
# Column name transformation/cleaning
df_car.columns = df_car.columns.str.lower().str.replace(' ', '_')
df_car.columns

Index(['make', 'model', 'year', 'engine_fuel_type', 'engine_hp',
       'engine_cylinders', 'transmission_type', 'driven_wheels',
       'number_of_doors', 'market_category', 'vehicle_size', 'vehicle_style',
       'highway_mpg', 'city_mpg', 'popularity', 'price', 'above_average'],
      dtype='object')

In [73]:
df_car.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [74]:
subset = ['Make',
          'Model',
          'Year',
          'Engine HP',
          'Engine Cylinders',
          'Transmission Type',
          'Vehicle Style',
          'highway MPG',
          'city mpg',
          'MSRP']

categorical_columns = list(df_car.dtypes[df_car.dtypes == 'object'].index)
for col in categorical_columns:
    df_car[col] = df_car[col].str.lower().str.replace(' ', '_')
    df_car[col] = df_car[col].fillna(0)

new_car = df_car[subset]

In [81]:
accuracies = []
global_accuracy = accuracy

for col in select_columns:
    subset = select_columns.copy()
    subset.remove(col)

    train_dict = X_train[subset].to_dict(orient='records')
    val_dict = y_val[subset].to_dict(orient='records')

    dv.fit(train_dict)

    X_train = dv.transform(train_dict)
    X_val = dv.transform(val_dict)


    model = LogisticRegression(solver='liblinear', C=1.0, random_state=42, max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    score = accuracy_score(y_val, y_pred)
    print(
        f"Column: {col} - Score: {round(score, 4)} - Diff: {round(global_accuracy-score, 5)}")
    accuracies.append(score)

KeyError: ignored


### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn.
* We'll need to use the original column `price`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data with a solver `'sag'`. Set the seed to `42`.
* This model also has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`.
* Round your RMSE scores to 3 decimal digits.

Which of these alphas leads to the best RMSE on the validation set?

- 0
- 0.01
- 0.1
- 1
- 10

> **Note**: If there are multiple options, select the smallest `alpha`.

In [46]:
from math import sqrt
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

new_car['price'] = np.log1p(df_car['price'])

In [49]:
# Split the data into training and validation sets
X = new_car[numerical]
y = new_car['above_average']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
scores = {}

for alpha  in [0, 0.01, 0.1, 1, 10]:
    ridge = Ridge(alpha=alpha, solver='sag', random_state=42)
    ridge.fit(X_train, y_train)
    y_val_pred = ridge.predict(X_val)
    rmse = sqrt(mean_squared_error(y_val, y_val_pred))
    scores[alpha] = round(rmse, 3)

for alpha, rmse in scores.items():
    print(f'{alpha}: {rmse}')

0: 0.323
0.01: 0.323
0.1: 0.323
1: 0.323
10: 0.323


In [51]:
scores.values()

dict_values([0.323, 0.323, 0.323, 0.323, 0.323])

In [52]:
print(f'Q6 Answer: 0')

Q6 Answer: 0
